In [1]:
import pandas as pd
from main import parse_current_state_log,parse_order_log, extract_position_events, extract_execution_details,load_price_data,build_trades_from_events,match_executions_to_trades,extract_trading_moments,merge_trades_with_real_pnl,flag_trades


In [2]:
# Paths
current_state_log_path = "binance_bot_data/current_state_new.log"
order_log_path = "binance_bot_data/order_new.log"
pnl_real_csv = "binance_bot_data/pnl_real_new.csv"
price_path = "binance_bot_data/price_data_new.csv"

first_trade_date = "2025-03-06 00:00:00"
first_exit_time = "2025-03-06 15:50:32"
last_exit_time = "2025-03-10 08:57:03"
merge_time_window = "10min"
output_trades = "df_trades_new.csv"
plot_output = "cumulative_pnl_comparison_new.png"

# Parse logs
df_log = parse_current_state_log(current_state_log_path)
df_log = df_log[df_log["timestamp"] >= pd.Timestamp(first_trade_date)].copy()
print("Current state log dataframe:")
print(df_log.head())
print("=" * 60)

df_order_log = parse_order_log(order_log_path)
print("Order log dataframe:")
print(df_order_log.head())
print("=" * 60)

Current state log dataframe:
                   timestamp  \
1 2025-03-06 02:14:23.921000   
2 2025-03-06 02:14:23.921000   
3 2025-03-06 15:49:25.980200   
4 2025-03-06 15:51:29.415900   
5 2025-03-06 15:53:33.630400   

                                           positions  
1  {'MEWUSDT': -1, 'TONUSDT': 1, 'ETCUSDT': -1, '...  
2  {'MEWUSDT': -1, 'TONUSDT': 1, 'ETCUSDT': -1, '...  
3                  {'FLOWUSDT': 1, 'RENDERUSDT': -1}  
4  {'FLOWUSDT': 1, 'RENDERUSDT': -1, 'MANTAUSDT':...  
5  {'FLOWUSDT': 1, 'RENDERUSDT': -1, 'MANTAUSDT':...  
Order log dataframe:
                timestamp                                            message
0 2025-03-06 15:49:28.883  INFO;broker_web_binancefut_mel_cm2;splitting o...
1 2025-03-06 15:49:29.404  INFO;broker_web_binancefut_mel_cm2;Preparing o...
2 2025-03-06 15:49:29.404  INFO;broker_web_binancefut_mel_cm2;Converted 6...
3 2025-03-06 15:49:29.405  INFO;broker_web_binancefut_mel_cm2;Received mi...
4 2025-03-06 15:49:29.405  INFO;broker_web

In [3]:
# Extract execution details
df_executions = extract_execution_details(df_order_log)
print("Execution details dataframe:")
print(df_executions)
print("=" * 60)

df_events = extract_position_events(df_log)
print("Extracted events dataframe:")
print(df_events)
df_events.to_csv("events.csv", index=True)
print("=" * 60)


Execution details dataframe:
                  timestamp        pair  execution_price
0   2025-03-06 15:50:32.716    FLOWUSDT          0.44600
1   2025-03-06 15:50:32.716  RENDERUSDT          4.07500
2   2025-03-06 15:52:05.575   MANTAUSDT          0.30650
3   2025-03-06 15:52:05.575     CHZUSDT          0.05028
4   2025-03-06 15:54:08.845     POLUSDT          0.25381
..                      ...         ...              ...
107 2025-03-10 04:15:28.523    MINAUSDT          0.25890
108 2025-03-10 07:47:04.207    NEARUSDT          2.57200
109 2025-03-10 07:47:04.207     FETUSDT          0.54250
110 2025-03-10 08:57:03.499     SEIUSDT          0.19090
111 2025-03-10 08:57:03.499     APEUSDT          0.50550

[112 rows x 3 columns]
Extracted events dataframe:
        symbol                  timestamp action  old_position  new_position
0      ETCUSDT 2025-03-06 02:14:23.921000   open             0            -1
1      LTCUSDT 2025-03-06 02:14:23.921000   open             0             1
2   

In [4]:

df_prices = load_price_data(price_path)
print("Prices dataframe:")
print(df_prices.head())
print("=" * 60)

df_trades = build_trades_from_events(df_events, df_prices)
print("Built trades dataframe:")
print(df_trades.head())
print("=" * 60)

# Match execution prices to trades
df_trades = match_executions_to_trades(df_trades, df_executions, time_tolerance="1min")
print("Trades with execution prices:")
print(df_trades.head())
print("=" * 60)

# Filter trades
first_exit = pd.Timestamp(first_exit_time)
last_exit = pd.Timestamp(last_exit_time)
df_trades = df_trades[
    (pd.to_datetime(df_trades["exit_time"]) >= first_exit) &
    (pd.to_datetime(df_trades["exit_time"]) <= last_exit)
].copy()
df_trades.set_index("exit_time", inplace=True)
df_trades.to_csv(output_trades, index=True)
print("Filtered trades saved:")
print(df_trades.head(20))
print("=" * 60)

df_real = extract_trading_moments(pnl_real_csv)
print("Filtered real pnl dataframe:")
print(df_real)
print("=" * 60)

df_pnl = pd.read_csv("pnl.csv", index_col=0)
print(df_pnl["value"])



Prices dataframe:
                     MINAUSDT_open  MINAUSDT_high  MINAUSDT_low  \
date                                                              
2025-03-05 23:03:00         0.3001         0.3001        0.2994   
2025-03-05 23:06:00         0.2995         0.2996        0.2989   
2025-03-05 23:09:00         0.2989         0.2995        0.2989   
2025-03-05 23:12:00         0.2993         0.2993        0.2987   
2025-03-05 23:15:00         0.2987         0.2993        0.2987   

                     MINAUSDT_close  MINAUSDT_volume  BCHUSDT_open  \
date                                                                 
2025-03-05 23:03:00          0.2996          32228.0        395.60   
2025-03-05 23:06:00          0.2989          17314.0        394.47   
2025-03-05 23:09:00          0.2993          17990.0        393.78   
2025-03-05 23:12:00          0.2987           7986.0        394.25   
2025-03-05 23:15:00          0.2992          12676.0        393.67   

                     

KeyError: 'exit_spread'

In [ ]:
df_merged = merge_trades_with_real_pnl(df_trades, df_real, time_window=merge_time_window)
print("Merged trades with real pnl:")
print(df_merged)
print("=" * 60)
df_merged.to_csv("pnl_comparison.csv", index=True)

# plot_cumulative_pnl(df_merged, df_pnl, output_path=plot_output)
flagged_calc_real, flagged_real_theo = flag_trades(df_merged, df_pnl, tolerance_diff=0.001)
print("Flagged trades (Calculated PnL vs Real PnL):")
print(flagged_calc_real[["exit_time", "symbol", "diff_calc_real", "diff_hours"]])
print("=" * 60)
print("Flagged trades (Real PnL vs Theoretical PnL):")
print(flagged_real_theo[["exit_time", "symbol", "diff_real_theo", "diff_hours"]])